In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pickle
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import cross_val_score,KFold
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import RANSACRegressor,ElasticNet


import warnings


warnings.filterwarnings("ignore")

# environment settings
data_path = 'Data/'

# Deserialize previously saved data from "preprocessing"
with open(data_path+'train_pp.obj', 'rb') as train_pp, \
open(data_path+'test_pp.obj','rb') as test_pp:
    train_df = pickle.load(train_pp)
    test_df = pickle.load(test_pp)
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])

In [2]:
X = train_df.loc[:,'Id':'2ndFlrSF-Sq']
y = train_df['SalePrice']
print("Shape of training set {}.\nShape of test set {}".format(X.shape,y.shape))

Shape of training set (1456, 349).
Shape of test set (1456,)


In [3]:
scaler = RobustScaler()
X_scaled=scaler.fit(X).transform(X)
test_scaled=scaler.transform(test_df)

In [4]:
rs = RANSACRegressor(base_estimator = ElasticNet(alpha=0.00065,l1_ratio=.55))

kf = KFold(10, shuffle=True, random_state=42).get_n_splits(X)
rmse_cv_rs= np.sqrt(-cross_val_score(rs, X, y, scoring="neg_mean_squared_error", cv = kf))
print("The 10-fold crossvalidation RMSE of Ransac is {:.5f} +/- {:.3f}".format(rmse_cv_rs.mean(),
                                                                               rmse_cv_rs.std()))

The 10-fold crossvalidation RMSE of Ransac is 0.11071 +/- 0.015


The 10-fold crossvalidation RMSE of XGB is 0.11064 +/- 0.015

In [ ]:
#Prediction on real test set using Ransac
rs.fit(X,y)
pred_results =rs.predict(test_df)
pred_results = np.expm1(pred_results)
result_df = pd.DataFrame(data={'Id': test_df["Id"].values,
                               'SalePrice': pred_results})
#Create output csv file
result_df.to_csv(data_path+"outputs/rs", index=False)